In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

ANALISI CULTURA E SPETTACOLO SENZA PESI

Utilizzeremo i seguenti dati: 

1. **Musei**
2. **Cinema**
3. **Teatri**

Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

Carichiamo il Grafo di base della rete stradale/ciclabile

In [2]:
# Carichiamo Graph Ciclabili
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "rb") as f:
    G_strade_ciclabili = pickle.load(f)

In [3]:
G_compose = G_strade_ciclabili.copy()

Rimuoviamo i pesi dalle strade

In [4]:

for u, v, k in G_compose.edges(keys=True):
    if G_compose[u][v][k]["tipo"] == "Strade_ciclabili":
        G_compose[u][v][k]['weight'] = 1

### Aggiunta di **musei** come **"poi"**

1. Carichiamo il file dei musei da "clean"
2. Utilizziamo **"connect_poi_nodes_to_graph()"** per connettere il geodataframe come "poi" al grafo delle strade/ciclabili.

In [5]:
gdf_musei = gpd.read_file(PATH_MUSEI_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_musei,
                                                       tipo = "Musei",
                                                       poi = True,
                                                       artificial = True)

### Aggiunta di **Teatri** come **"poi"**

In [6]:
gdf_teatri = gpd.read_file(PATH_TEATRI_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_teatri,
                                                       tipo = "Teatri",
                                                       poi = True,
                                                       artificial = True)

### Aggiunta di **Cinema** come **"poi"**

In [7]:
gdf_cinema = gpd.read_file(PATH_CINEMA_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_cinema,
                                                       tipo = "Cinema", 
                                                       poi = True,
                                                       artificial = True)

## Utilizziamo algoritmo di **Steiner Tree**

In [8]:
poi_nodes = [n for n, d in G_compose.nodes(data=True) if d.get("poi") == True]
G_cultura_e_spettacollo = nx.MultiDiGraph(steiner_tree(G_compose, terminal_nodes=poi_nodes, weight='weight')).to_undirected()

Risultato lo salviamo in Clean di Analisi (per ora anche il pickle in staging) in geojson pronto da visualizzare

In [9]:
with open("../Data/Staging/Analisi/Cultura_e_spettacolo_nopesi.pickle", "wb") as f:
    pickle.dump(G_cultura_e_spettacollo, f)

gdf_steiner = ox.graph_to_gdfs(G_cultura_e_spettacollo, edges=True, nodes=False)
gdf_steiner.to_file("../Data/Clean/Analisi/Cultura_e_spettacolo_nopesi.geojson", driver="GeoJSON")